In [1]:
import sys
from collections import defaultdict

class MCODE():
    def __init__(self, filename, weight_threshold=0.2):
        self.weight_threshold = 1 - weight_threshold
        self.filename = filename
        self.clusters = []

    def cluster(self):
        edges = defaultdict(set)

        # Citanje liste grana
        with open(self.filename, 'r') as f:
            for line in f:
                pair = line.split()
                a = pair[0]
                b = pair[2]
                edges[a].add(b)
                edges[b].add(a)
        print ('## Input graph loaded; %i nodes' % (len(edges),))

        # Lista klastera
        clusters = []

        # Faza 1: Odredjivanje tezina cvorova
        print ('## Weighting vertices...')
        weights = dict((v, 1.) for v in edges)
        for i, v in enumerate(edges):
            neighborhood = set((v,)) | edges[v]
            # ako cvor ima samo jednog suseda, onda znamo sve sto treba da znamo
            if len(neighborhood) <= 2:
                continue

            # provera da li vece k-jezgro postoji
            k = 1  # najvece validno k-jezgro
            while neighborhood:
                k_core = neighborhood.copy()
                invalid_nodes = True
                while invalid_nodes and neighborhood:
                    invalid_nodes = set(n for n in neighborhood if len(
                        edges[n] & neighborhood) <= k)
                    neighborhood -= invalid_nodes
                k += 1  # na kraju, k ce biti za jedan vece od onoga sta zelimo
                
            # tezina cvora = k-jezgro * gustina k-jezgra
            weights[v] = (k - 1) * (sum(len(edges[n] & k_core)
                                        for n in k_core) / (2. * len(k_core)**2))

        # Faza 2: Predikcija Molekularnih Kompleksa
        print('## Molecular complex prediction...')
        unvisited = set(edges)
        num_clusters = 0

        for seed in sorted(weights, key=weights.get, reverse=True):
            if seed not in unvisited:
                continue

            cluster, frontier = set((seed,)), set((seed,))
            w = weights[seed] * self.weight_threshold
            while frontier:
                cluster.update(frontier)
                unvisited -= frontier
                frontier = set(n for n in set.union(
                    *(edges[n] for n in frontier)) & unvisited if weights[n] > w)

            # Odsecanje: cuvamo samo komplekse 2-jezgra
            invalid_nodes = True
            while invalid_nodes and cluster:
                invalid_nodes = set(
                    n for n in cluster if len(edges[n] & cluster) < 2)
                cluster -= invalid_nodes

            if cluster:
                print (' '.join(cluster))
                num_clusters += 1
                print (num_clusters, len(cluster), seed)
                clusters.append(cluster)

        self.clusters = clusters

    def save_clusters(self, filehandle):
        with open(filehandle, 'w') as fh:
            for c in self.clusters:
                fh.write(' '.join(c) + "\n")
                


Ovde ucitavamo mrezu proteina i zatim pokrecemo "MCODE" algoritam koji izlistava klastere.

In [2]:
mcode = MCODE("files/original_network.sif")

In [3]:
mcode.cluster()

## Input graph loaded; 6008 nodes
## Weighting vertices...
## Molecular complex prediction...
CTRO_HUMAN RS3_HUMAN RL23_HUMAN NR2C2_HUMAN TRI25_HUMAN RS16_HUMAN RLA1_HUMAN RL32_HUMAN RS11_HUMAN PRKN_HUMAN RS15_HUMAN HEXI1_HUMAN RS8_HUMAN RL14_HUMAN RL38_HUMAN RL6_HUMAN RL23A_HUMAN HMGB1_HUMAN WWP2_HUMAN RS13_HUMAN RS3A_HUMAN RS28_HUMAN RS15A_HUMAN FBXW7_HUMAN RS4X_HUMAN PRC1_HUMAN RL30_HUMAN RL10A_HUMAN RL31_HUMAN RS14_HUMAN UBL4A_HUMAN KIF14_HUMAN RS24_HUMAN RSSA_HUMAN RS2_HUMAN RC3H1_HUMAN RL10_HUMAN KIF23_HUMAN RS23_HUMAN RL27A_HUMAN BTF3_HUMAN RL8_HUMAN RLA2_HUMAN RS18_HUMAN RL12_HUMAN RS21_HUMAN RL35A_HUMAN MEPCE_HUMAN FINC_HUMAN RL22_HUMAN PP1G_HUMAN RL15_HUMAN RC3H2_HUMAN RS9_HUMAN RS12_HUMAN CUL1_HUMAN BIRC3_HUMAN RL34_HUMAN RL9_HUMAN RL24_HUMAN RL36_HUMAN RL21_HUMAN RS20_HUMAN RS25_HUMAN RL11_HUMAN RLA0_HUMAN AAR2_HUMAN ECT2_HUMAN FZR1_HUMAN RL19_HUMAN RL27_HUMAN CYLD_HUMAN RS6_HUMAN UFL1_HUMAN RL13_HUMAN RL13A_HUMAN RS27_HUMAN RL7A_HUMAN RL5_HUMAN RL7_HUMAN RS5_HUMAN CUL3_HUM

In [4]:
network = set()
with open("files/original_network.sif", 'r') as file:
        for line in file:
            nodes = line.strip().split("\t-\t")
            if len(nodes) == 2:
                network.add(tuple(nodes))

Naredna funkcija cuva zadat kluster u datoteku na zadatoj putanji u formatu mreze.

In [5]:
def save_to_sif(path, cluster):
    cluster = set(cluster.split(' '))
    lines = []
    
    for edge in network:
        if edge[0] in cluster and edge[1] in cluster:
            lines.append(edge[0] + "\t-\t" + edge[1])
    
    with open(path, 'w') as file:
        for i, line in enumerate(lines):
            if i < len(lines) - 1:
                file.write(f"{line}\n")
            else:
                file.write(line)

In [7]:
commonGenesH = ["AIP","LEP","CMGA","CDN2A","CAV1","CTNB1",
                "MDR1","CASR","ALBU","G6PD","FLNA","ESR1","KLOT",
                "ACTB","MEN1","P53","VGFR2","EGFR","CDN2B",
                "EDNRB","VDR","LEG3","LRP5","SYT1","AMPE","MK01",
                "CCND1","TNF10","PTHY","KAP0","S12A3","POTEF"] 
for i in range(32):
    commonGenesH[i] += "_HUMAN"

In [9]:
cluster_contain_gene = {}
i = 0 
with open("files/modules_from_original_network.txt","r") as file:
    for line in file:
        i += 1
        if i == 16:
            break 
        
        file_name = "module_" + str(i) + ".sif" 
        cluster_contain_gene[file_name] = False
        
        for gene in commonGenesH:
            if line.find(gene) != -1:
                cluster_contain_gene[file_name] = True
                break
        
        save_to_sif("files/mcode_results/sif_files_for_clusters/" + file_name, line)

Naredni kod pokrece "MCODE" algoritam za 15 najboljih klastera dobijenih iz prethodno pokrenutog "MCODE" algoritmai potom cuva sve dobijene klastere.

In [11]:
import os

for file in os.listdir("files/mcode_results/sif_files_for_clusters/"):
        file_path = os.path.join("files/mcode_results/sif_files_for_clusters/", file)
        
        if cluster_contain_gene[file]:
            mcode = MCODE(file_path)
            mcode.cluster()
            mcode.save_clusters("files/mcode_results/auxilary_files/" + file)

## Input graph loaded; 95 nodes
## Weighting vertices...
## Molecular complex prediction...
DDX5_HUMAN NTRK1_HUMAN IF4A3_HUMAN ESR1_HUMAN EWS_HUMAN RFA1_HUMAN VIR_HUMAN SFPQ_HUMAN HNRPK_HUMAN HNRH1_HUMAN LMNA_HUMAN ROA3_HUMAN HNRPD_HUMAN DHX9_HUMAN LAP2B_HUMAN SNW1_HUMAN SRRM2_HUMAN CTNB1_HUMAN GRP78_HUMAN LAP2A_HUMAN DDX17_HUMAN
1 21 HNRPK_HUMAN
ROA2_HUMAN RALY_HUMAN RBMX_HUMAN HNRPC_HUMAN CDK9_HUMAN DHX15_HUMAN RBP56_HUMAN TIF1B_HUMAN ROA0_HUMAN SRSF3_HUMAN RFA2_HUMAN TERA_HUMAN HSP7C_HUMAN TOP1_HUMAN
2 14 ROA2_HUMAN
SF3B1_HUMAN CMTR1_HUMAN RT05_HUMAN SNUT1_HUMAN RT27_HUMAN PABP4_HUMAN NEK4_HUMAN SF3B2_HUMAN MOV10_HUMAN RT34_HUMAN
3 10 SF3B1_HUMAN
PTCD3_HUMAN DHX30_HUMAN EFTU_HUMAN
4 3 CD81_HUMAN
EF2_HUMAN EF1A1_HUMAN TBG1_HUMAN ARAF_HUMAN ARRB2_HUMAN RACK1_HUMAN
5 6 TBG1_HUMAN
STAU2_HUMAN TDRD3_HUMAN RBMS1_HUMAN
6 3 H12_HUMAN
## Input graph loaded; 384 nodes
## Weighting vertices...
## Molecular complex prediction...
SP20H_HUMAN SUPT3_HUMAN IN80E_HUMAN ST65G_HUMAN NFRKB_HUMAN TAF9_H

Naredni kod ucitava samo one prethodno dobijene klastere koje sadrze bar jedan zajednicki protein i pravi podmrezu na osnovu tih klastera da bi oni mogli kasnije da se prikazu u alatu "Cytoscape".

In [14]:
for f in os.listdir("files/mcode_results/auxilary_files/"):
    file_path = os.path.join("files/mcode_results/auxilary_files/", f)
    
    with open(file_path,"r") as file:
        i = 1
        for line in file:

            file_name = f[:-4] + "." + str(i) + ".sif"
            cluster_contain_gene[file_name] = False

            for gene in commonGenesH:
                if line.find(gene) != -1:
                    cluster_contain_gene[file_name] = True
                    i += 1
                    break
                    
            
            if cluster_contain_gene[file_name]:
                save_to_sif("files/mcode_results/sif_files_for_clusters/" + file_name, line)

Naredna funkcija pronalazi ime gena.

In [51]:
from Bio import ExPASy
from Bio import SwissProt
import time

def search_uniprot(query):
    handle = ExPASy.get_sprot_raw(query)
    record = SwissProt.read(handle)
    return record

U sledecem delu koda smo za svaki dobijeni podmodul, koji je sacuvan kao mreza (izlaz dela koda oznacen sa [14]), pronalazi imena na osnovu prethodne funkcije i cuva imena u posebnoj datoteci za svaki modul. Te datoteke se kasnije koriste za "enrichGO" i "enrichPathway" analizu.

In [52]:
for f in os.listdir("files/mcode_results/sif_files_for_clusters/"):
    if len(f) > 13:
        file_path = os.path.join("files/mcode_results/sif_files_for_clusters/", f)
        with open(file_path,"r") as file:
            genes = []
            for edge in file:
                g = edge.split("\t-\t")
                if g[1][-1] == '\n':
                    g[1] = g[1][:-1]
                if g[0] not in genes:
                    genes.append(g[0])
                if g[1] not in genes:
                    genes.append(g[1])
            line = ""
            start = time.time()
            for gene in genes:
                line = line + search_uniprot(gene).gene_name[0]['Name'].split(" ")[0] + " "
            line = line[:-1]
            print(f"{len(genes)} - {time.time()-start}")

            with open(file_path[:-4]+".txt","w") as file_to_write:
                file_to_write.write(line)

34 - 71.21146893501282
20 - 24.028204202651978
22 - 72.49398064613342
16 - 45.106850147247314
7 - 15.853612184524536
6 - 24.42577075958252
26 - 41.63715720176697
25 - 72.68410730361938
23 - 48.590914726257324
43 - 69.71946668624878
33 - 37.73845720291138
